Step 1️⃣ — Transformer Encoder (BERT-like)

In [1]:
import torch
import torch.nn as nn
import math

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, max_len, d_model]

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


Step 2️⃣ — Encoder Layer (Self-Attention + FFN)

In [2]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, dim_ff, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        # Self-Attention
        attn_output, _ = self.self_attn(x, x, x, attn_mask=src_mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Feed Forward
        ff_output = self.linear2(torch.relu(self.linear1(x)))
        x = self.norm2(x + self.dropout(ff_output))
        return x


Step 3️⃣ — Transformer Encoder (Stack Layers)

In [3]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, n_heads=8, num_layers=6, dim_ff=2048, dropout=0.1, max_len=512):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, n_heads, dim_ff, dropout) for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, input_ids, src_mask=None):
        x = self.token_emb(input_ids) * math.sqrt(self.d_model)
        x = self.pos_enc(x)
        x = x.transpose(0, 1)  # [seq_len, batch, d_model]
        for layer in self.layers:
            x = layer(x, src_mask)
        return x.transpose(0, 1)  # [batch, seq_len, d_model]


Step 4️⃣ — Add MLM Head (optional)

In [4]:
class BertLikeModel(nn.Module):
    def __init__(self, vocab_size, d_model=512, n_heads=8, num_layers=6, dim_ff=2048, dropout=0.1, max_len=512):
        super().__init__()
        self.encoder = TransformerEncoder(vocab_size, d_model, n_heads, num_layers, dim_ff, dropout, max_len)
        self.mlm_head = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids, src_mask=None):
        encoder_output = self.encoder(input_ids, src_mask)
        mlm_logits = self.mlm_head(encoder_output)
        return mlm_logits


Step 5️⃣ — Test the Model

In [5]:
vocab_size = 10000
seq_len = 16
batch_size = 2

model = BertLikeModel(vocab_size)
input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))

out = model(input_ids)
print(out.shape)  # [batch_size, seq_len, vocab_size]


torch.Size([2, 16, 10000])
